# Join EPC, deprivation and flood risk data
EPC data: https://epc.opendatacommunities.org/domestic/search  
Deprivation data: https://imd-by-postcode.opendatacommunities.org/imd/2019  
Flood risk data: https://www.getthedata.com/open-flood-risk-by-postcode  
  
Use the postcode previously placed into the Zoopla dataset to join EPC, deprivation and flood risk data. Since EPC is at the property (rather than postcode) level, use the average EPC for that postcode, road and property type

In [1]:
import os
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_columns', 100)

### Read in Zoopla, EPC, deprivation and flood risk files

In [2]:
AREA = 'Hinckley'
DATA_RAW_FOLDER = os.path.join('data', 'raw')
DATA_PROCESSED_FOLDER = os.path.join('data', 'processed')
SAVE_FOLDER = DATA_PROCESSED_FOLDER

In [3]:
if AREA == 'Nuneaton':
    zoopla_df_filename = 'zoopla_properties_with_postcode_nuneaton.csv'
    epc_df_filename = 'epcs_nuneaton.csv'
    deprivation_df_filename = '2019-deprivation-by-postcode_nuneaton.csv'
elif AREA == 'Hinckley':
    zoopla_df_filename = 'zoopla_properties_with_postcode_hinckley.csv'
    epc_df_filename = 'epcs_hinckley.csv'
    deprivation_df_filename = '2019-deprivation-by-postcode_hinckley.csv'
    
floodrisk_df_filename = 'open_flood_risk_by_postcode.csv'

zoopla_df = pd.read_csv(os.path.join(DATA_PROCESSED_FOLDER, zoopla_df_filename), dtype=str)
epc_df = pd.read_csv(os.path.join(DATA_RAW_FOLDER, epc_df_filename), dtype=str)
deprivation_df = pd.read_csv(os.path.join(DATA_RAW_FOLDER, deprivation_df_filename), dtype=str)
floodrisk_df = pd.read_csv(os.path.join(DATA_RAW_FOLDER, floodrisk_df_filename), header=None, dtype=str)

In [4]:
display(zoopla_df.head())
display(epc_df.head())
display(deprivation_df.head())
display(floodrisk_df.head())

,details_url,agent_phone,description,agent_address,latitude,longitude,outcode,country_code,num_bathrooms,listing_status,property_type,listing_id,num_recepts,post_town,displayable_address,floor_plan,image_url,street_name,agent_name,county,price_modifier,first_published_date,country,last_published_date,price,category,num_bedrooms,agent_logo,postcode,parish
0,https://www.zoopla.co.uk/for-sale/details/6388...,01455 886081,Attractive extended traditional bay fronted s...,"98 Castle Street, Hinckley",52.538830000000004,-1.396291,LE10,gb,1,sale,Semi-detached house,63884099,2,Hinckley,"Langdale Road, Hinckley LE10",NaN,https://lid.zoocdn.com/354/255/c9c518b9e9bd024...,Langdale Road,Scrivins & Co Estate Agents & Letting Agents,Leicestershire,NaN,2023-02-04 11:38:12,England,2023-02-04 11:39:32,260000.0,Residential,3,https://st.zoocdn.com/zoopla_static_agent_logo...,LE10 0NS,"Hinckley and Bosworth, unparished area"
1,https://www.zoopla.co.uk/for-sale/details/6387...,01455 364814,** viewing essential ** A beautifully present...,"112 Castle Street, Hinckley",52.548297999999996,-1.353169,LE10,gb,4,sale,Detached house,63878342,2,Hinckley,"Bradgate Gardens, Hinckley LE10",NaN,https://lid.zoocdn.com/354/255/e2c80f945cd69da...,Bradgate Gardens,Castle Estates,Leicestershire,offers_over,2023-02-03 15:54:45,England,2023-02-04 09:42:14,450000.0,Residential,5,https://st.zoocdn.com/zoopla_static_agent_logo...,LE10 1ND,"Hinckley and Bosworth, unparished area"
2,https://www.zoopla.co.uk/for-sale/details/6387...,01455 364814,**viewing essential ** A well appointed semi ...,"112 Castle Street, Hinckley",52.552856000000006,-1.3755549999999999,LE10,gb,1,sale,Semi-detached house,63874929,2,Hinckley,"York Road, Hinckley LE10",NaN,https://lid.zoocdn.com/354/255/4fd00c679828a04...,York Road,Castle Estates,Leicestershire,offers_over,2023-02-03 11:20:13,England,2023-02-03 13:17:45,280000.0,Residential,3,https://st.zoocdn.com/zoopla_static_agent_logo...,LE10 0RH,"Hinckley and Bosworth, unparished area"
3,https://www.zoopla.co.uk/for-sale/details/6387...,01455 364026,An immaculately maintained tastefully decorat...,"28-30 New Buildings, Hinckley",52.534348,-1.3928639999999999,LE10,gb,1,sale,Semi-detached house,63871971,1,Hinckley,"Strathmore Road, Hinckley LE10",NaN,https://lid.zoocdn.com/354/255/31105260d73c332...,Strathmore Road,Profiles,Leicestershire,offers_in_region_of,2023-02-02 22:07:06,England,2023-02-02 22:07:06,260000.0,Residential,3,https://st.zoocdn.com/zoopla_static_agent_logo...,LE10 0LW,"Hinckley and Bosworth, unparished area"
4,https://www.zoopla.co.uk/for-sale/details/6152...,01455 364871,You're sure to be impressed when you enter thi...,"84 Castle Steet, Hinckley",52.546017000000006,-1.38644,LE10,gb,1,sale,Bungalow,61524804,1,Hinckley,"Aulton Way, Hinckley, Leicestershire LE10",NaN,https://lid.zoocdn.com/354/255/88ccf710c285162...,Aulton Way,Your Move - Hinckley,Leicestershire,NaN,2023-02-01 18:08:21,England,2023-02-01 18:08:21,315000.0,Residential,3,https://st.zoocdn.com/zoopla_static_agent_logo...,LE10 0XB,"Hinckley and Bosworth, unparished area"


,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,PROPERTY_TYPE,BUILT_FORM,INSPECTION_DATE,LOCAL_AUTHORITY,CONSTITUENCY,COUNTY,LODGEMENT_DATE,TRANSACTION_TYPE,ENVIRONMENT_IMPACT_CURRENT,ENVIRONMENT_IMPACT_POTENTIAL,ENERGY_CONSUMPTION_CURRENT,ENERGY_CONSUMPTION_POTENTIAL,CO2_EMISSIONS_CURRENT,CO2_EMISS_CURR_PER_FLOOR_AREA,CO2_EMISSIONS_POTENTIAL,LIGHTING_COST_CURRENT,LIGHTING_COST_POTENTIAL,HEATING_COST_CURRENT,HEATING_COST_POTENTIAL,HOT_WATER_COST_CURRENT,HOT_WATER_COST_POTENTIAL,TOTAL_FLOOR_AREA,ENERGY_TARIFF,MAINS_GAS_FLAG,FLOOR_LEVEL,FLAT_TOP_STOREY,FLAT_STOREY_COUNT,MAIN_HEATING_CONTROLS,MULTI_GLAZE_PROPORTION,GLAZED_TYPE,GLAZED_AREA,EXTENSION_COUNT,NUMBER_HABITABLE_ROOMS,NUMBER_HEATED_ROOMS,LOW_ENERGY_LIGHTING,NUMBER_OPEN_FIREPLACES,HOTWATER_DESCRIPTION,HOT_WATER_ENERGY_EFF,HOT_WATER_ENV_EFF,FLOOR_DESCRIPTION,FLOOR_ENERGY_EFF,FLOOR_ENV_EFF,WINDOWS_DESCRIPTION,WINDOWS_ENERGY_EFF,WINDOWS_ENV_EFF,WALLS_DESCRIPTION,WALLS_ENERGY_EFF,WALLS_ENV_EFF,SECONDHEAT_DESCRIPTION,SHEATING_ENERGY_EFF,SHEATING_ENV_EFF,ROOF_DESCRIPTION,ROOF_ENERGY_EFF,ROOF_ENV_EFF,MAINHEAT_DESCRIPTION,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATCONT_DESCRIPTION,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_DESCRIPTION,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,WIND_TURBINE_COUNT,HEAT_LOSS_CORRIDOR,UNHEATED_CORRIDOR_LENGTH,FLOOR_HEIGHT,PHOTO_SUPPLY,SOLAR_WATER_HEATING_FLAG,MECHANICAL_VENTILATION,ADDRESS,LOCAL_AUTHORITY_LABEL,CONSTITUENCY_LABEL,POSTTOWN,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE
0,188e4e7604368b7386e5ff93771a037ccfb150c2861096...,5 Brockey Close,Barwell,NaN,LE9 8BG,10003465551,D,B,68,87,Bungalow,Detached,2022-10-04,E07000132,E14000583,Leicestershire,2022-10-10,Stock condition survey,67,87,270,102,2.2,48,0.9,44,44,418,370,59,39,47,Single,Y,NaN,NaN,NaN,NaN,100,double glazing installed before 2002,Normal,0,3,3,100,0,From main system,Good,Good,"Solid, no insulation (assumed)",NaN,NaN,Fully double glazed,Average,Average,"Cavity wall, as built, insulated (assumed)",Good,Good,"Room heaters, mains gas",NaN,NaN,"Pitched, 270 mm loft insulation",Good,Good,"Boiler and radiators, mains gas",Good,Good,"Programmer, room thermostat and TRVs",Good,Good,Low energy lighting in all fixed outlets,Very Good,Very Good,mains gas (not community),0,NaN,NaN,2.3,0.0,N,natural,"5 Brockey Close, Barwell",Hinckley and Bosworth,Bosworth,LEICESTER,England and Wales: 1983-1990,2022-10-10 19:12:48,Rented (social),6,NaN,100032074592,Energy Assessor
1,849605851212012102416531797929305,"2, Nob Hill",Norton juxta Twycross,NaN,CV9 3QE,3560172078,D,C,67,80,House,Detached,2012-10-19,E07000132,E14000583,Leicestershire,2012-10-24,marketed sale,60,75,156,94,8.4,34,5.2,112,112,1463,1067,172,105,243,Single,N,NODATA!,NaN,NaN,2106,85,"double glazing, unknown install date",Normal,1,8,8,73,0,From main system,Good,Average,"Solid, no insulation (assumed)",NaN,NaN,Mostly double glazing,Poor,Poor,"Cavity wall, as built, partial insulation (ass...",Average,Average,"Room heaters, wood logs",NaN,NaN,"Pitched, 150 mm loft insulation",Good,Good,"Boiler and radiators, oil",Good,Good,"Programmer, room thermostat and TRVs",Good,Good,Low energy lighting in 73% of fixed outlets,Very Good,Very Good,oil (not community),0,NO DATA!,NaN,NaN,0.0,NaN,natural,"2, Nob Hill, Norton juxta Twycross",Hinckley and Bosworth,Bosworth,ATHERSTONE,England and Wales: 1976-1982,2012-10-24 16:53:17,owner-occupied,26,19,100030495131,Address Matched
2,1062831709962013121817485087588537,"21, Barrie Road",NaN,NaN,LE10 0QU,3806087178,E,B,53,82,House,End-Terrace,2013-12-18,E07000132,E14000583,Leicestershire,2013-12-18,assessment for green deal,49,81,303,103,4.9,58,1.7,80,49,892,519,82,58,84,Single,Y,NODATA!,NaN,NaN,2107,100,"double glazing, unknown install date",Normal,1,5,5,36,0,From main system,Good,Good,"Suspended, no insulat

,Postcode,Postcode Status,LSOA code,LSOA Name,User Data A,User Data B,User Data C,User Data D,User Data E,User Data F,User Data G,User Data H,User Data I,User Data J,User Data K,User Data L,User Data M,User Data N,User Data O,User Data P,Index of Multiple Deprivation Rank,Index of Multiple Deprivation Decile,Income Rank,Income Decile,Income Score,Employment Rank,Employment Decile,Employment Score,Education and Skills Rank,Education and Skills Decile,Health and Disability Rank,Health and Disability Decile,Crime Rank,Crime Decile,Barriers to Housing and Services Rank,Barriers to Housing and Services Decile,Living Environment Rank,Living Environment Decile,IDACI Rank,IDACI Decile,IDACI Score,IDAOPI Rank,IDAOPI Decile,IDAOPI Score
0,Postcode,**UNMATCHED**,NaN,NaN,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,Ward,Parish,Introduced,Terminated,Altitude,Country,Last Updated,Quality,LSOA Code,LSOA Name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CV10 0RY,Live,E01025881,Hinckley and Bosworth 008D E01025881,Yes,52.559033,-1.482983,435148,295814,SP351958,Twycross and Witherley with Sheepy,Witherley,1980-01-01,NaN,81,England,2022-11-25,Within the building of the matched address clo...,E01025881,Hinckley and Bosworth 008D,23101,8,25980,8,0.051,25261,8,0.048,24502,8,26276,9,19219,6,2985,1,19542,6,24795,8,0.065,25661,8,0.068
2,CV10 0SB,Live,E01025881,Hinckley and Bosworth 008D E01025881,Yes,52.56227,-1.487188,434860,296172,SP348961,Twycross and Witherley with Sheepy,Witherley,1980-01-01,NaN,83,England,2022-11-25,Within the building of the matched address clo...,E01025881,Hinckley and Bosworth 008D,23101,8,25980,8,0.051,25261,8,0.048,24502,8,26276,9,19219,6,2985,1,19542,6,24795,8,0.065,25661,8,0.068
3,CV10 0TT,Live,E01025818,Hinckley and Bosworth 008B E01025818,Yes,52.554782,-1.461007,436641,295352,SP366953,Ambien,Higham on the Hill,1980-01-01,NaN,96,England,2022-11-25,Within the building of the matched address clo...,E01025818,Hinckley and Bosworth 008B,15723,5,21828,7,0.068,21111,7,0.061,21116,7,21057,7,20760,7,2591,1,2324,1,24953,8,0.064,19878,7,0.101
4,CV10 0TU,Live,E01025818,Hinckley and Bosworth 008B E01025818,Yes,52.553685,-1.464958,436374,295228,SP363952,Ambien,Higham on the Hill,1998-12-01,NaN,90,England,2022-11-25,Within the building of the matched address clo...,E01025818,Hinckley and Bosworth 008B,15723,5,21828,7,0.068,21111,7,0.061,21116,7,21057,7,20760,7,2591,1,2324,1,24953,8,0.064,19878,7,0.101


,0,1,2,3,4,5,6,7,8,9
0,AL10 0AA,\N,None,\N,\N,\N,522503,208775,51.764264,-0.226254
1,AL10 0AB,\N,None,\N,\N,\N,522680,209765,51.773122,-0.223341
2,AL10 0AD,\N,None,\N,\N,\N,522997,209812,51.773475,-0.218732
3,AL10 0AE,\N,None,\N,\N,\N,522530,209750,51.773020,-0.225519
4,AL10 0AG,\N,None,\N,\N,\N,522515,209794,51.773419,-0.225721


### Fix column names in each dataset

In [5]:
# Deprivation dataset - colnames spread over two rows.
# If colname starts with 'User Data', then use second row, else first row

for colname in deprivation_df.columns:
    if colname.startswith('User Data'):
        new_colname = deprivation_df[colname].iloc[0]
        deprivation_df.rename(columns={colname: new_colname}, inplace=True)

# remove the first row
deprivation_df = deprivation_df.iloc[1:]
deprivation_df.head()

,Postcode,Postcode Status,LSOA code,LSOA Name,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,Ward,Parish,Introduced,Terminated,Altitude,Country,Last Updated,Quality,LSOA Code,LSOA Name,Index of Multiple Deprivation Rank,Index of Multiple Deprivation Decile,Income Rank,Income Decile,Income Score,Employment Rank,Employment Decile,Employment Score,Education and Skills Rank,Education and Skills Decile,Health and Disability Rank,Health and Disability Decile,Crime Rank,Crime Decile,Barriers to Housing and Services Rank,Barriers to Housing and Services Decile,Living Environment Rank,Living Environment Decile,IDACI Rank,IDACI Decile,IDACI Score,IDAOPI Rank,IDAOPI Decile,IDAOPI Score
1,CV10 0RY,Live,E01025881,Hinckley and Bosworth 008D E01025881,Yes,52.559033,-1.482983,435148,295814,SP351958,Twycross and Witherley with Sheepy,Witherley,1980-01-01,NaN,81,England,2022-11-25,Within the building of the matched address clo...,E01025881,Hinckley and Bosworth 008D,23101,8,25980,8,0.051,25261,8,0.048,24502,8,26276,9,19219,6,2985,1,19542,6,24795,8,0.065,25661,8,0.068
2,CV10 0SB,Live,E01025881,Hinckley and Bosworth 008D E01025881,Yes,52.56227,-1.487188,434860,296172,SP348961,Twycross and Witherley with Sheepy,Witherley,1980-01-01,NaN,83,England,2022-11-25,Within the building of the matched address clo...,E01025881,Hinckley and Bosworth 008D,23101,8,25980,8,0.051,25261,8,0.048,24502,8,26276,9,19219,6,2985,1,19542,6,24795,8,0.065,25661,8,0.068
3,CV10 0TT,Live,E01025818,Hinckley and Bosworth 008B E01025818,Yes,52.554782,-1.461007,436641,295352,SP366953,Ambien,Higham on the Hill,1980-01-01,NaN,96,England,2022-11-25,Within the building of the matched address clo...,E01025818,Hinckley and Bosworth 008B,15723,5,21828,7,0.068,21111,7,0.061,21116,7,21057,7,20760,7,2591,1,2324,1,24953,8,0.064,19878,7,0.101
4,CV10 0TU,Live,E01025818,Hinckley and Bosworth 008B E01025818,Yes,52.553685,-1.464958,436374,295228,SP363952,Ambien,Higham on the Hill,1998-12-01,NaN,90,England,2022-11-25,Within the building of the matched address clo...,E01025818,Hinckley and Bosworth 008B,15723,5,21828,7,0.068,21111,7,0.061,21116,7,21057,7,20760,7,2591,1,2324,1,24953,8,0.064,19878,7,0.101
5,CV10 0TZ,Live,E01025818,Hinckley and Bosworth 008B E01025818,Yes,52.54929,-1.449379,437434,294747,SP374947,Ambien,Higham on the Hill,1980-01-01,NaN,104,England,2022-11-25,Within the building of the matched address clo...,E01025818,Hinckley and Bosworth 008B,15723,5,21828,7,0.068,21111,7,0.061,21116,7,21057,7,20760,7,2591,1,2324,1,24953,8,0.064,19878,7,0.101


In [6]:
# Flood risk dataset - set names
colnames = {0: 'postcode', 1: 'FID', 2: 'PROB_4BAND', 3: 'SUITABILITY', 4: 'PUB_DATE',
            5: 'RISK_FOR_INSURANCE_SOP', 6: 'easting', 7: 'northing', 8: 'latitude', 9: 'longitude'}

floodrisk_df.rename(columns=colnames, inplace=True)
floodrisk_df.head()

,postcode,FID,PROB_4BAND,SUITABILITY,PUB_DATE,RISK_FOR_INSURANCE_SOP,easting,northing,latitude,longitude
0,AL10 0AA,\N,None,\N,\N,\N,522503,208775,51.764264,-0.226254
1,AL10 0AB,\N,None,\N,\N,\N,522680,209765,51.773122,-0.223341
2,AL10 0AD,\N,None,\N,\N,\N,522997,209812,51.773475,-0.218732
3,AL10 0AE,\N,None,\N,\N,\N,522530,209750,51.773020,-0.225519
4,AL10 0AG,\N,None,\N,\N,\N,522515,209794,51.773419,-0.225721


### Get street name from EPC dataset

In [7]:
def get_street_name(address_1, address_2):
    
    """
    Get street name from first two street address fields
    """
    
    street_and_road = re.compile(r'^\d,\s+')
    
    # if street name starts with a number (maybe followed by comma) and a space, likely next part is street name
    if street_and_road.match(address_1):
        street = re.split(street_and_road, address_1)[1].lower()
        
    # otherwise choose the second part of the address as the street name
    else:
        street = str(address_2).lower()
        
    return street

In [8]:
epc_df['Street'] = epc_df[['ADDRESS1', 'ADDRESS2']].apply(lambda x: get_street_name(x[0], x[1]), axis=1)

### Also remove prefix 'England and Wales: ' from construction age band in EPC data, and drop imvalid values

In [9]:
epc_df['CONSTRUCTION_AGE_BAND'].value_counts(dropna=False)

England and Wales: 1950-1966       5445
NO DATA!                           5351
England and Wales: 1967-1975       5160
England and Wales: 1930-1949       4618
England and Wales: 1900-1929       4223
England and Wales: 1983-1990       3541
England and Wales: 1976-1982       3190
England and Wales: before 1900     2384
England and Wales: 1996-2002       2264
England and Wales: 2003-2006       2126
England and Wales: 1991-1995       1825
England and Wales: 2007 onwards    1011
NaN                                 340
England and Wales: 2007-2011        334
2021                                186
2022                                169
2020                                146
INVALID!                            100
England and Wales: 2012 onwards      74
2018                                 73
2019                                 54
2017                                 26
2016                                 17
1930                                  6
Name: CONSTRUCTION_AGE_BAND, dtype: int6

In [10]:
# e.g. England and Wales: 1967-1975 -> 1967-1975
epc_df['CONSTRUCTION_AGE_BAND'] = epc_df['CONSTRUCTION_AGE_BAND'].replace('England and Wales: ' , '', regex=True)
epc_df['CONSTRUCTION_AGE_BAND'] = epc_df['CONSTRUCTION_AGE_BAND'].replace(r'(NO DATA!|INVALID!)' , np.nan, regex=True)

In [11]:
epc_df['CONSTRUCTION_AGE_BAND'].value_counts(dropna=False)

NaN             5791
1950-1966       5445
1967-1975       5160
1930-1949       4618
1900-1929       4223
1983-1990       3541
1976-1982       3190
before 1900     2384
1996-2002       2264
2003-2006       2126
1991-1995       1825
2007 onwards    1011
2007-2011        334
2021             186
2022             169
2020             146
2012 onwards      74
2018              73
2019              54
2017              26
2016              17
1930               6
Name: CONSTRUCTION_AGE_BAND, dtype: int64

### Select out relevant columns in each dataset and drop duplicates

In [12]:
zoopla_df = zoopla_df[['listing_id', 'parish', 'post_town', 'postcode', 'street_name', 'latitude', 'longitude',
                       'property_type', 'num_bedrooms', 'num_bathrooms', 'description',
                       'first_published_date', 'last_published_date', 'price', 'price_modifier']].drop_duplicates()

epc_df = epc_df[['Street', 'POSTCODE', 'CURRENT_ENERGY_RATING', 'POTENTIAL_ENERGY_RATING', 
                 'CURRENT_ENERGY_EFFICIENCY', 'POTENTIAL_ENERGY_EFFICIENCY',
                 'PROPERTY_TYPE', 'BUILT_FORM', 'TOTAL_FLOOR_AREA', 
                 'NUMBER_HABITABLE_ROOMS', 'CONSTRUCTION_AGE_BAND']].drop_duplicates()
epc_df.rename(columns={'POSTCODE': 'postcode'}, inplace=True)

deprivation_df = deprivation_df[['Postcode', 'Index of Multiple Deprivation Decile', 'Income Decile',
                                 'Employment Decile', 'Education and Skills Decile',
                                 'Health and Disability Decile', 'Crime Decile', 
                                 'Barriers to Housing and Services Decile', 'Living Environment Decile',
                                 'IDACI Decile', 'IDAOPI Decile']].drop_duplicates()

floodrisk_df = floodrisk_df[['postcode', 'PROB_4BAND']].drop_duplicates()

### Map PROPERTY_TYPE and BUILT_FORM in EPC data onto those for property_type in Zoopla data
We will then be able to narrow down possible EPC data houses onto the Zoopla house and join them

In [13]:
# first see if EPC appears in any of the Zoopla property descriptions
zoopla_df_epcs = zoopla_df[zoopla_df['description'].str.contains('epc', case=False)]['description']
for epc in zoopla_df_epcs:
    print(epc)

A deceptively spacious terraced home in the heart of Hinckley. Conveniently located, surprisingly large, and well presented throughout. EPC E
Here is a most attractive modern Semi Detached House offering well planned accommodation considered ideal for a young growing family. EPC rating D.


Very few descriptions have EPCs so use the EPC data to infer them

In [14]:
print('Zoopla house types:')
display(zoopla_df['property_type'].value_counts())

print('\nEPC house types:')
display(epc_df.groupby(['PROPERTY_TYPE', 'BUILT_FORM'])['PROPERTY_TYPE'].count())

Zoopla house types:


Semi-detached house       100
Detached house             88
Terraced house             28
Flat                       28
Town house                 12
Detached bungalow          11
End terrace house           6
Bungalow                    5
Maisonette                  4
Link-detached house         4
Block of flats              1
Semi-detached bungalow      1
Name: property_type, dtype: int64


EPC house types:


PROPERTY_TYPE  BUILT_FORM          
Bungalow       Detached                 2979
               Enclosed End-Terrace        2
               Enclosed Mid-Terrace        2
               End-Terrace               395
               Mid-Terrace               484
               NO DATA!                    2
               Semi-Detached            1659
Flat           Detached                  803
               Enclosed End-Terrace      153
               Enclosed Mid-Terrace       97
               End-Terrace               883
               Mid-Terrace              1136
               NO DATA!                  182
               Semi-Detached            1419
House          Detached                 9405
               Enclosed End-Terrace       96
               Enclosed Mid-Terrace       26
               End-Terrace              2784
               Mid-Terrace              4977
               NO DATA!                  188
               Semi-Detached           11663
Maisonette     Deta

In [15]:
def set_property_genre_epc(property_type, built_form):
    
    """
    Map the EPC property type and built form pairs into the Zoopla values
    """
    
    property_type = str(property_type).lower()
    built_form = str(built_form).lower()
    
    if property_type == 'bungalow':
        property_genre = 'Bungalow'
            
    elif property_type == 'flat':
        property_genre = 'Flat'
        
    elif property_type == 'house':
        if built_form == 'detached':
            property_genre = 'Detached house'
        elif built_form == 'semi-detached':
            property_genre = 'Semi-detached house'
        elif 'end-terrace' in built_form:
            property_genre = 'End terrace house'
        elif 'mid-terrace' in built_form:
            property_genre = 'Terraced house'
        else:
            property_genre = 'House'
            
    elif property_type == 'maisonette':
        property_genre = 'Maisonette'
        
    else:
        property_genre = 'Other/Unknown'
        
    return property_genre


In [16]:
def set_property_genre_zoopla(property_type):
    
    """
    Merge rare Zoopla property types into their own parent category or an 'Other' genre
    This is to create a temporary property type field that we can then join on to the EPC data
    But for the prediction, we will use the original property types
    """
    
    property_type = str(property_type).lower()
    
    if property_type == 'town house':
        property_genre = 'Terraced house'
    elif property_type == 'link-detached house':
        property_genre = 'Detached house'
    elif 'bungalow' in property_type:
        property_genre = 'Bungalow'
    elif property_type in ['semi-detached house', 'detached house', 'bungalow', 'flat',
       'town house', 'link-detached house', 'end terrace house', 'terraced house',
       'detached bungalow', 'maisonette', 'semi-detached bungalow']:
        property_genre = property_type.capitalize()
    else:
        property_genre = 'Other/Unknown'
        
    return property_genre
    

In [17]:
epc_df['property_type_temp'] = epc_df[['PROPERTY_TYPE', 'BUILT_FORM']].apply(
    lambda x: set_property_genre_epc(x[0], x[1]), axis=1)

zoopla_df['property_type_temp'] = zoopla_df['property_type'].apply(set_property_genre_zoopla)

display(epc_df['property_type_temp'].value_counts())
display(zoopla_df['property_type_temp'].value_counts())

Semi-detached house    11663
Detached house          9405
Bungalow                5523
Terraced house          5003
Flat                    4673
End terrace house       2880
Maisonette               574
House                    188
Other/Unknown              3
Name: property_type_temp, dtype: int64

Semi-detached house    100
Detached house          92
Terraced house          40
Flat                    28
Bungalow                17
End terrace house        6
Other/Unknown            5
Maisonette               4
Name: property_type_temp, dtype: int64

Now for a given postcode and property type in the EPC dataset, get both the most common EPC and range of EPCs

In [19]:
epc_df = epc_df.groupby(by=['postcode', 'property_type_temp']).agg({
    'Street': pd.Series.mode,
    'CURRENT_ENERGY_RATING': [pd.Series.mode, 'min', 'max'],
    'POTENTIAL_ENERGY_RATING': [pd.Series.mode, 'min', 'max'],
    'CURRENT_ENERGY_EFFICIENCY': [pd.Series.mode, 'min', 'max'],
    'POTENTIAL_ENERGY_EFFICIENCY': [pd.Series.mode, 'min', 'max'],
    'TOTAL_FLOOR_AREA': pd.Series.mode,
    'NUMBER_HABITABLE_ROOMS': pd.Series.mode,
    'CONSTRUCTION_AGE_BAND': pd.Series.mode
})

In [20]:
epc_df.head(10)

Street CURRENT_ENERGY_RATING      \
                                              mode                  mode min   
postcode property_type_temp                                                    
CV10 0RY Detached house            atherstone road                     C   C   
CV10 0SB Bungalow                  kenilworth farm                     C   C   
         Detached house             watling street                     F   F   
CV10 0TT Detached house             watling street                     F   F   
         Semi-detached house            mira drive                [D, E]   D   
CV10 0TZ Flat                 youth with a mission                     C   C   
CV13 0AA Bungalow                     shenton lane                     F   E   
CV13 0AD Detached house                ambion lane                     F   F   
CV13 0AE Detached house              sutton cheney                     E   E   
CV13 0AG Bungalow                      main street                     E   D   

                                 POTENTIAL_ENERGY_RATING          \
                             max                    mode min max   
postcode property_type_temp                                        
CV10 0RY Detached house        C                       B   B   B   
CV10 0SB Bungalow              C                       A   A   A   
         Detached house        F                       B   B   B   
CV10 0TT Detached house        F                       D   D   D   
         Semi-detached house   E                       B   B   B   
CV10 0TZ Flat                  C                       C   C   C   
CV13 0AA Bungalow              G                       E   A   F   
CV13 0AD Detached house        F                       E   E   E   
CV13 0AE Detached house        E                       B   B   B   
CV13 0AG Bungalow              F                  [C, D]   C   D   

                             CURRENT_ENERGY_EFFICIENCY          \
                                                  mode min max   
postcode property_type_temp                                      
CV10 0RY Detached house                             72  72  72   
CV10 0SB Bungalow                                   74  73  78   
         Detached house                             34  34  34   
CV10 0TT Detached house                             35  35  35   
         Semi-detached house                  [51, 56]  51  56   
CV10 0TZ Flat                                       70  70  70   
CV13 0AA Bungalow                     [20, 31, 34, 48]  20  48   
CV13 0AD Detached house                             35  35  35   
CV13 0AE Detached house                             42  42  42   
CV13 0AG Bungalow                     [33, 48, 53, 67]  33  67   

                             POTENTIAL_ENERGY_EFFICIENCY            \
                                                    mode  min  max   
postcode property_type_temp                                          
CV10 0RY Detached house                               90   90   90   
CV10 0SB Bungalow                             [138, 139]  129  140   
         Detached house                               82   82   82   
CV10 0TT Detached house                               67   67   67   
         Semi-detached house                    [82, 85]   82   85   
CV10 0TZ Flat                                         78   78   78   
CV13 0AA Bungalow                       [23, 41, 53, 92]   23   92   
CV13 0AD Detached house                               54   54   54   
CV13 0AE Detached house                               88   88   88   
CV13 0AG Bungalow                       [58, 67, 74, 75]   58   75   

                               TOTAL_FLOOR_AREA NUMBER_HABITABLE_ROOMS  \
                                           mode                   mode   
postcode property_type_temp                                              
CV10 0RY Detached house                     202                      7   
CV10 0SB Bungalow                  [19, 20, 36]     

In [21]:
# set the indices back as columns so we can join them to the zoopla dataframe
epc_df = epc_df.reset_index()

In [22]:
epc_df.head()

postcode   property_type_temp           Street CURRENT_ENERGY_RATING      \
                                             mode                  mode min   
0  CV10 0RY       Detached house  atherstone road                     C   C   
1  CV10 0SB             Bungalow  kenilworth farm                     C   C   
2  CV10 0SB       Detached house   watling street                     F   F   
3  CV10 0TT       Detached house   watling street                     F   F   
4  CV10 0TT  Semi-detached house       mira drive                [D, E]   D   

      POTENTIAL_ENERGY_RATING         CURRENT_ENERGY_EFFICIENCY          \
  max                    mode min max                      mode min max   
0   C                       B   B   B                        72  72  72   
1   C                       A   A   A                        74  73  78   
2   F                       B   B   B                        34  34  34   
3   F                       D   D   D                        35  35  35   
4   E                       B   B   B                  [51, 56]  51  56   

  POTENTIAL_ENERGY_EFFICIENCY           TOTAL_FLOOR_AREA  \
                         mode  min  max             mode   
0                          90   90   90              202   
1                  [138, 139]  129  140     [19, 20, 36]   
2                          82   82   82              225   
3                          67   67   67              195   
4                    [82, 85]   82   85         [81, 89]   

  NUMBER_HABITABLE_ROOMS CONSTRUCTION_AGE_BAND  
                    mode                  mode  
0                      7             1967-1975  
1                     []                    []  
2                      7           before 1900  
3                      8             1950-1966  
4                      5             1950-1966

### Left join EPC dataset to Zoopla dataset
If postcode and property type same, then join

In [24]:
zoopla_df_test = zoopla_df.merge(epc_df, on=['postcode', 'property_type_temp'], how='left')
zoopla_df_test.head()

,listing_id,parish,post_town,postcode,street_name,latitude,longitude,property_type,num_bedrooms,num_bathrooms,description,first_published_date,last_published_date,price,price_modifier,property_type_temp,"(Street, mode)","(CURRENT_ENERGY_RATING, mode)","(CURRENT_ENERGY_RATING, min)","(CURRENT_ENERGY_RATING, max)","(POTENTIAL_ENERGY_RATING, mode)","(POTENTIAL_ENERGY_RATING, min)","(POTENTIAL_ENERGY_RATING, max)","(CURRENT_ENERGY_EFFICIENCY, mode)","(CURRENT_ENERGY_EFFICIENCY, min)","(CURRENT_ENERGY_EFFICIENCY, max)","(POTENTIAL_ENERGY_EFFICIENCY, mode)","(POTENTIAL_ENERGY_EFFICIENCY, min)","(POTENTIAL_ENERGY_EFFICIENCY, max)","(TOTAL_FLOOR_AREA, mode)","(NUMBER_HABITABLE_ROOMS, mode)","(CONSTRUCTION_AGE_BAND, mode)"
0,63884099,"Hinckley and Bosworth, unparished area",Hinckley,LE10 0NS,Langdale Road,52.538830000000004,-1.396291,Semi-detached house,3,1,Attractive extended traditional bay fronted s...,2023-02-04 11:38:12,2023-02-04 11:39:32,260000.0,NaN,Semi-detached house,nan,D,C,F,B,B,D,"[47, 62]",34,72,87,57,90,"[70, 87, 88]",5,1930-1949
1,63878342,"Hinckley and Bosworth, unparished area",Hinckley,LE10 1ND,Bradgate Gardens,52.548297999999996,-1.353169,Detached house,5,4,** viewing essential ** A beautifully present...,2023-02-03 15:54:45,2023-02-04 09:42:14,450000.0,offers_over,Detached house,bradgate gardens,C,C,D,B,B,C,76,66,78,"[77, 80, 84, 85]",75,86,"[108, 110, 123, 128, 129, 154, 160, 163, 164, ...",7,2007 onwards
2,63874929,"Hinckley and Bosworth, unparished area",Hinckley,LE10 0RH,York Road,52.552856000000006,-1.3755549999999999,Semi-detached house,3,1,**viewing essential ** A well appointed semi ...,2023-02-03 11:20:13,2023-02-03 13:17:45,280000.0,offers_over,Semi-detached house,nan,C,C,E,B,B,C,"[54, 70]",54,73,83,70,87,79,5,1950-1966
3,63871971,"Hinckley and Bosworth, unparished area",Hinckley,LE10 0LW,Strathmore Road,52.534348,-1.3928639999999999,Semi-detached house,3,1,An immaculately maintained tastefully decorat...,2023-02-02 22:07:06,2023-02-02 22:07:06,260000.0,offers_in_region_of,Semi-detached house,nan,D,C,F,B,B,E,"[52, 61]",24,72,85,54,89,103,5,1930-1949
4,61524804,"Hinckley and Bosworth, unparished area",Hinckley,LE10 0XB,Aulton Way,52.546017000000006,-1.38644,Bungalow,3,1,You're sure to be impressed when you enter thi...,2023-02-01 18:08:21,2023-02-01 18:08:21,315000.0,NaN,Bungalow,nan,C,C,D,B,B,D,"[63, 71, 72]",55,75,"[63, 70, 88, 89, 91]",63,91,"[51, 52]",2,1976-1982


### For a given postcode, street name, and house type, calculate the average EPC

### Save to csv file

In [ ]:
try:
    os.mkdir(SAVE_FOLDER)
except OSError:
    pass

save_file = os.path.join(SAVE_FOLDER, f'zoopla_properties_with_postcode_epc_dep_flood_{AREA.lower()}.csv')
    
#zoopla_df.to_csv(save_file, index=False)